In [1]:
# !sudo yum install Xvfb -y -y
# !pip install scrapy
# !pip install fake_useragent
# !pip install scrapy-selenium
# !pip install selenium==4.9.1
# !pip install boto3==1.28.5
# !pip install crochet
# !pip install awslambdaric
# !pip install botocore==1.31.10
# #

# import os
# cmd = '''
# sudo yum install atk cups-libs gtk3 libXcomposite alsa-lib libXcursor libXdamage libXext libXi libXrandr libXScrnSaver libXtst pango at-spi2-atk libXt xorg-x11-server-Xvfb xorg-x11-xauth dbus-glib dbus-glib-devel -y
# '''
# os.system(cmd)


# !curl  -Lo "/tmp/chromedriver.zip" "https://chromedriver.storage.googleapis.com/114.0.5735.90/chromedriver_linux64.zip"
# !unzip /tmp/chromedriver.zip -d /opt/ -y
# !sudo ln -sf /opt/chromedriver /usr/local/bin/chromedriver

# !curl -Lo "/tmp/chrome-linux.zip" "https://www.googleapis.com/download/storage/v1/b/chromium-browser-snapshots/o/Linux_x64%2F1135561%2Fchrome-linux.zip?alt=media"
# !unzip /tmp/chrome-linux.zip -d /opt/ -y
# !sudo ln -sf /opt/chrome-linux/chrome /usr/local/bin/chrome




Loaded plugins: dkms-build-requires, extras_suggestions, langpacks, priorities,
              : update-motd, versionlock
amzn2-core                                               | 3.7 kB     00:00     
amzn2extra-docker                                        | 3.0 kB     00:00     
amzn2extra-kernel-5.10                                   | 3.0 kB     00:00     
amzn2extra-python3.8                                     | 3.0 kB     00:00     
centos-extras                                            | 2.9 kB     00:00     
copr:copr.fedorainfracloud.org:vbatts:shadow-utils-newxi | 3.3 kB     00:00     
https://download.docker.com/linux/centos/2/x86_64/stable/repodata/repomd.xml: [Errno 14] HTTPS Error 404 - Not Found
Trying other mirror.
libnvidia-container/x86_64/signature                     |  833 B     00:00     
libnvidia-container/x86_64/signature                     | 2.1 kB     00:00 !!! 
neuron                                                   | 2.9 kB     00:00     
neuron/prima

https://download.docker.com/linux/centos/2/x86_64/stable/repodata/repomd.xml: [Errno 14] HTTPS Error 404 - Not Found
Trying other mirror.


61 packages excluded due to repository priority protections
Package atk-2.22.0-3.amzn2.0.2.x86_64 already installed and latest version
Package 1:cups-libs-1.6.3-51.amzn2.0.2.x86_64 already installed and latest version
Package gtk3-3.22.30-3.amzn2.x86_64 already installed and latest version
Package libXcomposite-0.4.4-4.1.amzn2.0.2.x86_64 already installed and latest version
Package alsa-lib-1.1.4.1-2.amzn2.x86_64 already installed and latest version
Package libXcursor-1.1.15-1.amzn2.x86_64 already installed and latest version
Package libXdamage-1.1.4-4.1.amzn2.0.2.x86_64 already installed and latest version
Package libXext-1.3.3-3.amzn2.0.2.x86_64 already installed and latest version
Package libXi-1.7.9-1.amzn2.0.2.x86_64 already installed and latest version
Package libXrandr-1.5.1-2.amzn2.0.3.x86_64 already installed and latest version
Package libXtst-1.2.3-1.amzn2.0.2.x86_64 already installed and latest version
Package pango-1.42.4-4.amzn2.x86_64 already installed and latest version


In [2]:
import json
import os, datetime, requests
import botocore
from fake_useragent import UserAgent
import boto3
import scrapy
from urllib.parse import urlparse
import hashlib
from scrapy.crawler import CrawlerRunner
from scrapy_selenium import SeleniumRequest
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as ec
from scrapy.utils.project import get_project_settings
from crochet import setup
import threading
from iopenmall import IOpenMallSpider
# from iopenmall_tier5 import IOpenMall_tier5 

setup()

ua = UserAgent()

In [3]:
class LambdaRunner:
    category_links = []

    def __init__(self):
        self.finished = threading.Event()
        self.results = []

    def run_spider(self):
        # Create a CrawlerRunner with project settings
        settings = get_project_settings()
        runner = CrawlerRunner(settings)

        # Create an instance of the spider class
        spider_cls = IOpenMallSpider

        # Callback function to handle the spider results
        def handle_results(result):
            self.results.append(result)

            # Check if the spider has finished running
            if len(self.results) == 1:
                self.finished.set()

        # Start the first spider run
        deferred = runner.crawl(spider_cls, self)
        deferred.addCallback(handle_results)

        # Start the reactor
        runner.join()

    def wait_for_completion(self):
        self.finished.wait()

    def get_results(self):
        return self.results
    
def handler(event, context):
    try:
        print("Starting tier2 links crawling")
        runner = LambdaRunner()
        runner.run_spider()
        runner.wait_for_completion()

        queue_list = []

        # Aggregate input JSON format
        for category_info in runner.category_links:
            category_link = category_info['link']
            tier1 = category_info['tier1'] # add on 08/30
            tier2 = category_info['tier2'] # add on 08/30
            tier3 = category_info['tier3'] # add on 08/30
            
            queue_list.append({'category_link': category_link, 
                              'tier1':tier1,  # add on 08/30
                              'tier2':tier2, # add on 08/30 
                              'tier3':tier3 }) # add on 08/30
        print(f'len(queue_list):{len(runner.category_links)}')
        return {
            'statusCode': 200,
            'body': {'message': queue_list}
        }
    except Exception as e:
        print(str(e))
        return {
            'statusCode': 500,
            'body': json.dumps({'error': str(e)})
        }


In [4]:
# headers = {"Content-Type": "application/x-www-form-urlencoded",
#            "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36"}
# response = requests.get('https://myip.com.tw/', headers=headers)

# # Check the response status code
# if response.status_code == 200:
#     ip = response.text[response.text.find("<font color=green>") + 18:response.text.find("</font></h1>")]
#     # Request was successful
#     print(ip)
# else:
#     # Request failed
#     print('Request failed with status code:', response.status_code)

In [5]:
handler("","")


Starting tier2 links crawling
Starting to parse
len(queue_list):282


{'statusCode': 200,
 'body': {'message': [{'category_link': 'https://mall.iopenmall.tw/iopen/index.php?action=store_product_sort&prod_sort_ou=1205',
    'tier1': '美妝保健',
    'tier2': '美妝保養',
    'tier3': '手足保養、美甲'},
   {'category_link': 'https://mall.iopenmall.tw/iopen/index.php?action=store_product_sort&prod_sort_ou=1226',
    'tier1': '美妝保健',
    'tier2': '美妝保養',
    'tier3': '美髮護理'},
   {'category_link': 'https://mall.iopenmall.tw/iopen/index.php?action=store_product_sort&prod_sort_ou=1233',
    'tier1': '美妝保健',
    'tier2': '美妝保養',
    'tier3': '男士保養'},
   {'category_link': 'https://mall.iopenmall.tw/iopen/index.php?action=store_product_sort&prod_sort_ou=1250',
    'tier1': '美妝保健',
    'tier2': '美妝保養',
    'tier3': '香水、香氛'},
   {'category_link': 'https://mall.iopenmall.tw/iopen/index.php?action=store_product_sort&prod_sort_ou=1251',
    'tier1': '美妝保健',
    'tier2': '美妝保養',
    'tier3': '彩妝'},
   {'category_link': 'https://mall.iopenmall.tw/iopen/index.php?action=store_product_sort

## tier 5

In [13]:
# from iopenmall_tier5 import IOpenMallSpider

In [18]:
timeout_seconds = float(os.getenv('TIMEOUT')) if os.getenv('TIMEOUT') is not None else 150  # seconds
class LambdaRunner:
    target_url = ""
    receipt_handle = ""
    timeout = True
    input_url = ""
    category_info = []
    
    def __init__(self, url):
        self.finished = threading.Event()
        self.results = []
        if url != "":
            self.target_url = url

    def run_spider(self):
        # Create a CrawlerRunner with project settings
        settings = get_project_settings()
        runner = CrawlerRunner(settings)

        # Create an instance of the spider class
        spider_cls = IOpenMallSpider

        # Callback function to handle the spider results
        def handle_results(result):
            self.results.append(result)

            # Check if the spider has finished running
            if len(self.results) == 1:
                self.finished.set()

        # Start the first spider run
        deferred = runner.crawl(spider_cls, url=self.target_url, runner=self)
        deferred.addCallback(handle_results)

        # Start the reactor
        runner.join()

    # Function to wait for spider completion
    def wait_for_completion(self):
        self.finished.wait(timeout=timeout_seconds)

    # Function to get spider results
    def get_results(self):
        return self.results

In [118]:
def check_tier(tier_dict: dict):
    if "tier5" in tier_dict:
        return "tier5"
    elif "tier4" in tier_dict:
        return "tier4"
    elif "tier3" in tier_dict:
        return "tier3"
    elif "tier2" in tier_dict:
        return "tier2"
    else:
        return "tier1"


def hash_function(original_string):
    # Create a hash object using SHA-256 algorithm
    hasher = hashlib.sha256()

    # Convert the input string to bytes (required by hashlib)
    input_bytes = original_string.encode('utf-8')

    # Update the hash object with the input bytes
    hasher.update(input_bytes)

    # Get the hexadecimal representation of the hash value
    return hasher.hexdigest()
    # Function to start pushing data to SQS and S3



class IOpenMallSpider(scrapy.Spider):
    name = "iopenmall"
    allowed_domains = ["mall.iopenmall.tw/iopen/"]
    start_urls = "https://mall.iopenmall.tw/iopen/"
    user_agent = ua.random
    receipt_handle = ""
    target_url = ""
    runner = None

    def __init__(self, url=None, runner=None, **kwargs):
        self.target_url = url
        self.runner = runner

        super(IOpenMallSpider, self).__init__(**kwargs)

    def error_handle(self, error):
        print(error)
        print(f'Invalid page: {self.target_url}')
        self.runner.timeout = True

    def start_requests(self):
        print("Starting queue_tier3_links from start_requests")
        print(f'self.target_url: {self.target_url}')
        headers = {"Host": urlparse(self.target_url).netloc,
                   'Accept-Encoding': 'gzip, deflate, br',
                   'Accept-Language': 'en-US,en;q=0.9,zh-TW;q=0.8,zh;q=0.7,zh-CN;q=0.6,ja;q=0.5',
                   'Sec-Fetch-Dest': 'document',
                   'Sec-Fetch-Mode': 'navigate',
                   'Sec-Fetch-Site': 'none',
                   'Upgrade-Insecure-Requests': '1',
                   "Referer": "https://mall.iopenmall.tw/iopen/",
                   "User-Agent": ua.random}
        yield SeleniumRequest(url=self.target_url,
                              headers=headers,
                              callback=self.parse_tier3,
                              wait_time=15,
                              errback=self.error_handle,
                              wait_until=ec.any_of(ec.presence_of_element_located((By.CLASS_NAME, 'first')),
                                                   ec.presence_of_element_located((By.ID, 'backgroundContent'))))

#     @staticmethod
#     def send_to_sqs(message):
#         try:
#             print(message['MessageGroupId'])
#             response = sqs.send_message(QueueUrl=queue_tier4_links,
#                                         MessageGroupId=message['MessageGroupId'],
#                                         MessageBody=message['MessageBody'],
#                                         MessageDeduplicationId=message['MessageGroupId'])
#             if response["ResponseMetadata"]["HTTPStatusCode"] != 200:
#                 print('Fail')
#         except Exception as e:
#             print(str(e))

#     @staticmethod
#     def send_to_s3(m, hash_id):
#         try:
#             json_file_name = f'{hash_id}.json'
#             response = s3.put_object(Bucket=s3_bucket_name, Key=json_file_name, Body=json.dumps(m, ensure_ascii=False))
#             # check if it's successful
#             if response["ResponseMetadata"]["HTTPStatusCode"] != 200:
#                 print('Fail')
#         except Exception as e:
#             print(str(e))

    def start_to_push(self, current_tier, hash_id, tier_dict, category_link, message_body):

        # modify on 08/30 
        message = {'Id': hash_id, 'MessageGroupId': hash_id,
                   'MessageBody': json.dumps({
                       'tier1_category_name': tier_dict["tier1"],
                       'tier2_category_name': tier_dict.get("tier2", ""),
                       'tier3_category_name': tier_dict.get("tier3", ""),
                       'tier4_category_name': tier_dict.get("tier4", ""),
                       'tier5_category_name': tier_dict.get("tier5", ""),
                       'category_link': category_link if not None else "",
                       'category_type': current_tier}, ensure_ascii=False)}
            
        print(f'currrnt_tier: {current_tier}, {tier_dict.get(current_tier, "")}')
        print()
        
#         thread_sqs = threading.Thread(target=self.send_to_sqs, args=(message,))
#         threads.append(thread_sqs)
#         thread_sqs.start()

#         self.send_to_sqs(message)
#         self.runner.category_info.append(message_body)

#         if export_to_s3 is not None and export_to_s3 == "On":
#             self.send_to_s3(message_body, hash_id)
#             thread_s3 = threading.Thread(target=self.send_to_s3, args=(message_body, hash_id))
#             threads.append(thread_s3)
#             thread_s3.start()

#         for thread in threads:
#             thread.join()

    # modify entire function 
    def parse_tier3(self, response):
        try:
            print("Starting to parse_tier3")
            first_menu_text = self.runner.tier1 # add on 08/30
            sec_menu_text = self.runner.tier2 # add on 08/30
            tri_menu_text = self.runner.tier3 # add on 08/30
            # print(first_menu_text, sec_menu_text, tri_menu_text)
            tri_menu_text = response.css('div.it913-default div span::text').get()
            for_menus = response.css('div.it913-default > ul > li')

            threads = []
            
            # modify on 08/30
            tier_dict = {}
            if len(for_menus) > 0:

                for for_menu in for_menus:
                    for_text = for_menu.css('div a::text').get()
                    for_link = for_menu.css('div a::attr(href)').get()
                    tier_dict = {}
                    fif_menus = for_menu.css('ul li div a')
                    if len(fif_menus) > 0: 

                        for fif_menu in fif_menus:
                            
                            fif_text = fif_menu.css('::text').get()
                            fif_link = fif_menu.css('::attr(href)').get()

                            tier_dict = {}
                            
                            link = f"https://mall.iopenmall.tw/iopen/{fif_link}"
                            tier_dict['tier5'] = fif_text
                            tier_dict['tier4'] = for_text     
                            tier_dict['tier3'] = tri_menu_text  
                            tier_dict['tier2'] = sec_menu_text
                            tier_dict['tier1'] = first_menu_text

                            self.parse_process(tier_dict, link) # change position on 08/30
                            # thread = threading.Thread(target=self.parse_process, args=(tier_dict, link))
                            # threads.append(thread)
                            # thread.start()
                    else:
                        tier_dict['tier4'] = for_text     
                        tier_dict['tier3'] = tri_menu_text  
                        tier_dict['tier2'] = sec_menu_text
                        tier_dict['tier1'] = first_menu_text
                        print(first_menu_text, sec_menu_text, tri_menu_text, for_text)
                        link = f"https://mall.iopenmall.tw/iopen/{for_link}"
                        
                        self.parse_process(tier_dict, link) # change position on 08/30
                        # thread = threading.Thread(target=self.parse_process, args=(tier_dict, link))
                        # threads.append(thread)
                        # thread.start()
            else:
                tier_dict['tier3'] = tri_menu_text   # add on 08/30
                tier_dict['tier2'] = sec_menu_text   # add on 08/30
                tier_dict['tier1'] = first_menu_text # add on 08/30
                link = response.url # add on 08/30
                
                self.parse_process(tier_dict, link)
                # thread = threading.Thread(target=self.parse_process, args=(tier_dict, link))
                # threads.append(thread)
                # thread.start()

            for thread in threads:
                thread.join()
        except Exception as e:
            print(e)
        finally:
            self.runner.timeout = False
    
    
    # modify the function 
    # Function to process individual category items
    def parse_process(self, tier_dict, category_link):

        current_tier = check_tier(tier_dict)
        
        # modify on 08/30
        if tier_dict is not None: 
            message_body = {'tier1_category_name': tier_dict.get("tier1", ""),
                            'tier2_category_name': tier_dict.get("tier2", ""),
                            'tier3_category_name': tier_dict.get("tier3", ""),
                            'tier4_category_name': tier_dict.get("tier4", ""),
                            'tier5_category_name': tier_dict.get("tier5", ""),
                            'category_link': category_link,
                            'category_type': current_tier}
        print(message_body)

        hash_id = hash_function(str(category_link))
        self.start_to_push(current_tier, hash_id, tier_dict, category_link, message_body)        


In [119]:

def handler(event, context):
    try:
        print(f"Starting to crawl: {datetime.datetime.now()}")
        times = 0
        if "statusCode" not in event:
            # Initialize the LambdaRunner with the provided category link

            # headers = {"Content-Type": "application/x-www-form-urlencoded",
            #            "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36"}
            # response = requests.get('https://myip.com.tw/', headers=headers)
            #
            # # Check the response status code
            # if response.status_code == 200:
            #     ip = response.text[response.text.find("<font color=green>") + 18:response.text.find("</font></h1>")]
            #     # Request was successful
            #     print(ip)
            # else:
            #     # Request failed
            #     print('Request failed with status code:', response.status_code)
            runner = LambdaRunner(event["category_link"])
            runner.input_url = event["category_link"]
            runner.tier1 = event["tier1"] # add on 08/30
            runner.tier2 = event["tier2"] # add on 08/30
            runner.tier3 = event["tier3"] # add on 08/30
            runner.run_spider()
            runner.wait_for_completion()

            print(f"End date and time:{datetime.datetime.now()}")

        else:
            times = int(event["times"])
            if times < 4:
                # Initialize the LambdaRunner with the provided category link
                runner = LambdaRunner(event["category_link"])
                runner.input_url = event["category_link"]
                runner.tier1 = event["tier1"]  # add on 08/30
                runner.tier2 = event["tier2"]  # add on 08/30
                runner.tier3 = event["tier3"]  # add on 08/30
                
                runner.run_spider()
                runner.wait_for_completion()
                print(f"End date and time:{datetime.datetime.now()}")
            else:
                return {
                    'statusCode': 429,
                    'body': "",
                    'times': times,
                    "category_link": event["category_link"]
                }

        times = times + 1
        if not runner.timeout:
            return {
                'statusCode': 200,
                'body': 'Completed!',
                'times': times,
                # "category_link": event["category_link"],
                # "category_info": runner.category_info
            }
        else:
            return {
                'statusCode': 408,
                'body': event["category_link"],
                'times': times,
                "category_link": event["category_link"]
            }
    except Exception as e:
        print(e)
        return {
            'statusCode': 500,
            'body': json.dumps({'error': str(e)})
        }
   

In [120]:
handler(  {'category_link': 'https://mall.iopenmall.tw/iopen/index.php?action=store_product_sort&prod_sort_ou=1031',
    'tier1': '流行時尚',
    'tier2': '男生衣著',
    'tier3': '上衣'}, "")



Starting to crawl: 2023-08-30 06:41:34.635338
Starting queue_tier3_links from start_requests
self.target_url: https://mall.iopenmall.tw/iopen/index.php?action=store_product_sort&prod_sort_ou=1031
Starting to parse_tier3
流行時尚 男生衣著 上衣 襯衫
{'tier1_category_name': '流行時尚', 'tier2_category_name': '男生衣著', 'tier3_category_name': '上衣', 'tier4_category_name': '襯衫', 'tier5_category_name': '', 'category_link': 'https://mall.iopenmall.tw/iopen/index.php?action=store_product_sort&prod_sort_ou=1032', 'category_type': 'tier4'}
currrnt_tier: tier4, 襯衫

流行時尚 男生衣著 上衣 Polo衫
{'tier1_category_name': '流行時尚', 'tier2_category_name': '男生衣著', 'tier3_category_name': '上衣', 'tier4_category_name': 'Polo衫', 'tier5_category_name': '', 'category_link': 'https://mall.iopenmall.tw/iopen/index.php?action=store_product_sort&prod_sort_ou=1033', 'category_type': 'tier4'}
currrnt_tier: tier4, Polo衫

流行時尚 男生衣著 上衣 T恤
{'tier1_category_name': '流行時尚', 'tier2_category_name': '男生衣著', 'tier3_category_name': '上衣', 'tier4_category_name'

{'statusCode': 200, 'body': 'Completed!', 'times': 1}